In [ ]:
import os
import pandas as pd
from autogluon.tabular import TabularPredictor 

In [4]:
presets = ['best_quality']
save_path = "./save/label1"
predictor = TabularPredictor(label=label,
                             path=save_path,
                             eval_metric='roc_auc')

In [ ]:


hyperparameters = {
                    'GBM': [
                         {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
                         # {},
                         # 'GBMLarge',
                    ],

                    'CAT': {},

                    'XGB': {},

                    # 'FASTAI': {},

                    'RF': [
                         {'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}},
                         # {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}},
                         # {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}},
                    ],

                    'XT': [
                          {'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}},
                    #     {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}},
                    #     {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}},
                    ],
                    
                    # 'KNN': [
                    #      {'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}},
                    #     {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}},
                    # ],
                }

# Ten-fold cross validation, multi-layer stacking
predictor.fit(train_data=train_data,
             #ag_args_fit={'num_gpus': 1}, 
             # tuning_data=val_data,
             time_limit=3000,
             presets=presets,
             # auto_stack=True,
             num_bag_sets=1,
             # use_bag_holdout=True,
             # holdout_frac = 0.3,
             num_stack_levels=1,
             num_bag_folds=10,
             hyperparameters=hyperparameters,
             )


# Test validation cohort
result = predictor.leaderboard(val_data, silent=True)
result.to_csv(os.path.join(save_path, "result.csv"), index=False)
result

In [ ]:
# Gain feature weighting and ranking
feature_importance = predictor.feature_importance(data=train_data, model='WeightedEnsemble_L2')